In [1]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')
from src.utils.logging_utils import initialize_logger
initialize_logger(logger, log_name="client")

print("Setup was successful.")

Setup was successful.


In [2]:
from src.utils.market_utils import best_ask, best_bid, spread, midquote, get_vitals
import numpy as np
import time

In [20]:
e = Exchange()
a = e.connect()


2021-01-24 12:29:17,946 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2021-01-24 12:29:17,949 [client    ] [Thread-5    ] background thread started [in /usr/local/lib/python3.6/site-packages/optibook/synchronous_wrapper.py:83]
2021-01-24 12:29:17,949 [client    ] [Thread-5    ] background thread started
2021-01-24 12:29:17,958 [client    ] [Thread-5    ] opened connection [in /usr/local/lib/python3.6/site-packages/optibook/base_client.py:180]
2021-01-24 12:29:17,958 [client    ] [Thread-5    ] opened connection
2021-01-24 12:29:17,962 [client    ] [Thread-5    ] start read <StreamReader t=<_SelectorSocketTransport fd=101 read=polling write=<idle, bufsize=0>>> [in /usr/local/lib/python3.6/site-packages/optibook/base_client.py:116]
2021-01-24 12:29:17,962 [client    ] [Thread-5    ] start read <StreamReader t=<_SelectorSocketTransport fd=101 read=polling write=<idle, bufsize=0>>>
2021-01-24 12:29:17,965 [client    ] [Thread-5    ] logged in! [in /usr/local/lib/python3.6/site-

In [21]:
e.get_pnl()

1342.6000000000063

2021-01-24 12:30:45,215 [client    ] [Thread-5    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time. [in /usr/local/lib/python3.6/site-packages/optibook/exchange_client.py:305]
2021-01-24 12:30:45,215 [client    ] [Thread-5    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [100]:
print(e.get_positions())

pos ={'PHILIPS_A': -30, 'PHILIPS_B': 20}
pos = e.get_positions()

while list(pos.values())!=[0,0]:
    
    for s, p in pos.items():
        price_book = e.get_last_price_book(s)
        # Check market vitals
        best_ask, best_bid, spread, midquote = get_vitals(price_book)
        #print(best_ask, best_bid, spread, midquote)
        if p > 0:
            volume= min(p, 5)
            price = best_bid
            print(f'{s}, {price}, {volume}, ask')
            e.insert_order(s, price=price, volume=volume, side='ask', order_type='ioc')
        elif p < 0:
            volume= min(-p, 5)
            price = best_ask
            e.insert_order(s, price=price, volume=volume, side='bid', order_type='ioc')
            print(f'{s}, {price}, {volume}, bid')
            
    # update pos
    pos = e.get_positions()
    print(f'end: {pos}')
    time.sleep(.01)
print(e.get_positions())

{'PHILIPS_A': 0, 'PHILIPS_B': 4}
PHILIPS_B, 81.10000000000001, 4, ask
end: {'PHILIPS_A': 0, 'PHILIPS_B': 3}
PHILIPS_B, 81.10000000000001, 3, ask
end: {'PHILIPS_A': 0, 'PHILIPS_B': 2}
PHILIPS_B, 81.0, 2, ask
end: {'PHILIPS_A': 0, 'PHILIPS_B': 0}
{'PHILIPS_A': 0, 'PHILIPS_B': 0}


2021-01-24 11:40:33,849 [client    ] [Thread-16   ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [103]:
for s in pos.items():
    print(s)

('PHILIPS_A', 0)
('PHILIPS_B', 0)


In [6]:
pos ={'PHILIPS_A': -30, 'PHILIPS_B': 20}

In [19]:
def hedge():
    pos = e.get_positions()
    pos_diff = pos['PHILIPS_A']+pos['PHILIPS_B']

    price_book = e.get_last_price_book('PHILIPS_A')
    best_ask, best_bid, spread, midquote = get_vitals(price_book) # always buy or sell in A

    if pos_diff > 0: # more in A, get rid of a
        side, price = 'ask', best_bid
        logger.debug(f'More in A {pos_diff}, getting rid {pos}')
    elif pos_diff < 0: # less in A, buy A
        side, price = 'bid', best_ask
        logger.debug(f'Less in A {pos_diff}, buying {pos}')
    else:
        return True

    volume= min(abs(pos_diff), 5)
    logger.debug(f'side : {side}, volume: {volume}, price {price}')
    
    e.insert_order('PHILIPS_A', price=price, volume=volume, side=side, order_type='ioc')
    return False


for i in range(5):
    #pos = e.get_positions()
    hedge()

2021-01-24 12:16:13,116 [client    ] [MainThread  ] Less in A -3, buying {'PHILIPS_A': -1, 'PHILIPS_B': -2} [in <ipython-input-19-f338aad83966>:13]
2021-01-24 12:16:13,116 [client    ] [MainThread  ] Less in A -3, buying {'PHILIPS_A': -1, 'PHILIPS_B': -2}
2021-01-24 12:16:13,124 [client    ] [MainThread  ] side : bid, volume: 3, price 79.2 [in <ipython-input-19-f338aad83966>:18]
2021-01-24 12:16:13,124 [client    ] [MainThread  ] side : bid, volume: 3, price 79.2
2021-01-24 12:16:13,140 [client    ] [Thread-4    ] Private trade: ( tradeId = 18085,
  timestamp = 1611490573128257773,
  instrumentId = "PHILIPS_A",
  orderId = 91264,
  price = 79.2,
  volume = 3,
  side = bid ). [in /usr/local/lib/python3.6/site-packages/optibook/exchange_client.py:153]
2021-01-24 12:16:13,140 [client    ] [Thread-4    ] Private trade: ( tradeId = 18085,
  timestamp = 1611490573128257773,
  instrumentId = "PHILIPS_A",
  orderId = 91264,
  price = 79.2,
  volume = 3,
  side = bid ).
2021-01-24 12:16:13,142 

In [ ]:
from src.utils.market_utils import 

In [18]:
e.insert_order('PHILIPS_A', price=78, volume=2, side='ask', order_type='ioc') # sell
e.insert_order('PHILIPS_B', price=78, volume=1, side='ask', order_type='ioc') # sell

2021-01-24 12:16:09,437 [client    ] [Thread-4    ] Private trade: ( tradeId = 18083,
  timestamp = 1611490569425949194,
  instrumentId = "PHILIPS_A",
  orderId = 91183,
  price = 78.9,
  volume = 2,
  side = ask ). [in /usr/local/lib/python3.6/site-packages/optibook/exchange_client.py:153]
2021-01-24 12:16:09,437 [client    ] [Thread-4    ] Private trade: ( tradeId = 18083,
  timestamp = 1611490569425949194,
  instrumentId = "PHILIPS_A",
  orderId = 91183,
  price = 78.9,
  volume = 2,
  side = ask ).
2021-01-24 12:16:09,441 [client    ] [Thread-4    ] trade end ( tradeId = 18083,
  timestamp = 1611490569425949194,
  instrumentId = "PHILIPS_A",
  orderId = 91183,
  price = 78.9,
  volume = 2,
  side = ask ) [in /usr/local/lib/python3.6/site-packages/optibook/exchange_client.py:299]
2021-01-24 12:16:09,441 [client    ] [Thread-4    ] trade end ( tradeId = 18083,
  timestamp = 1611490569425949194,
  instrumentId = "PHILIPS_A",
  orderId = 91183,
  price = 78.9,
  volume = 2,
  side = as

91184

In [46]:
list(pos.values())

[-1, 0]

In [27]:
pos = e.get_positions()

In [28]:
*pos.values()==0

SyntaxError: invalid syntax (<ipython-input-28-d7bcdcc176a0>, line 1)

In [24]:
list(pos.values())==[0,0]

True

In [30]:
{'A':-30, 'B':20}

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [34]:
e.get_positions()

{'PHILIPS_A': 0, 'PHILIPS_B': 0}